<a href="https://colab.research.google.com/github/rams1987/LMSYS-Human-preference-detection/blob/main/LMSYS_human_preference_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ramasamyseenivasagan","key":"f41bb0714d3baef12d51ba89162604ef"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c lmsys-chatbot-arena

 89% 51.0M/57.0M [00:01<00:00, 47.1MB/s]
100% 57.0M/57.0M [00:01<00:00, 48.8MB/s]


In [4]:
!unzip lmsys-chatbot-arena

Archive:  lmsys-chatbot-arena.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import os
import numpy as np
import pandas as pd
import spacy

In [6]:
data_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

data_df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [7]:
def determine_winner(row):
    if row['winner_model_a'] == 1:
        return 'model_a'
    elif row['winner_model_b'] == 1:
        return 'model_b'
    else:
        return 'tie'

data_df['winner'] = data_df.apply(determine_winner, axis=1)

data_df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,winner
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,model_a
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,model_b
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,tie
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,model_a
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,model_b


In [13]:
data_df.drop(columns=['id','winner_model_a', 'winner_model_b', 'winner_tie'], inplace=True)

data_df.head()

,model_a,model_b,prompt,response_a,response_b,winner
0,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",model_a
1,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",model_b
2,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",tie
3,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",model_a
4,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",model_b


In [ ]:
# prompt: Using dataframe data_df: code to train gradient boosting classifier

from sklearn.ensemble import GradientBoostingClassifier

# Create the classifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, subsample=0.5, random_state=0)

# Train the classifier
clf.fit(data_df.drop(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie'], axis=1), data_df['winner_tie'])

# Predict the class

In [ ]:
# prompt: prepare this data for gradient boosting classifier

import pandas as pd
# Separate features and target
X = data_df.drop('label', axis=1)
y = data_df['label']

# Encode categorical features
X = pd.get_dummies(X)

# Split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
# prompt: how to use gradient boosting for the data_df

from sklearn.ensemble import GradientBoostingClassifier

# Separate features and target
X = data_df.drop('label', axis=1)
y = data_df['label']

# Train the model
model = GradientBoostingClassifier()
model.fit(X, y)

# Make predictions
predictions = model.predict(test_df)
